In [1]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [4]:
import os
project_drive_path = "/content/drive/MyDrive/Colab_Notebooks/malay-qa-bot-rag" # Verify path
%cd {project_drive_path}
print(f"Current directory: {os.getcwd()}")
!ls -a # Check files

/content/drive/MyDrive/Colab_Notebooks/malay-qa-bot-rag
Current directory: /content/drive/MyDrive/Colab_Notebooks/malay-qa-bot-rag
 01-FineTuning-QA.ipynb       faiss_malay_ecommerce_kb_index   knowledge_base
'02-RAG-Pipeline (1).ipynb'   .git			       malay-qa-model-finetuned
 02-RAG-Pipeline.ipynb	      .gitignore		       requirements.txt
 app.py			      .ipynb_checkpoints	       squad_format_qa_pairs.json


In [12]:
# --- POC A Code ---
from sentence_transformers import SentenceTransformer
import torch
import numpy as np # Import numpy for checking shape


embedding_model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
print(f"POC A: Loading Sentence Transformer: {embedding_model_name}")


try:
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Using device: {device}")
    embedding_model = SentenceTransformer(embedding_model_name, device=device)
    print("Embedding model loaded successfully.")


    malay_sentences = [
        "Apakah polisi pemulangan Lazada?",
        "Penghantaran mengambil masa 3-5 hari bekerja.",
        "ShopeePay adalah dompet digital."
    ]
    print(f"\nEncoding {len(malay_sentences)} sample sentences...")
    embeddings_array = embedding_model.encode(malay_sentences) # Returns numpy array


    print(f"Embeddings generated.")
    print(f"Shape: {embeddings_array.shape}") # Should be (3, 384) for this model
    print(f"Type: {type(embeddings_array)}")
    print(f"First 5 dims of first embedding: {embeddings_array[0][:5]}")
except Exception as e:
    print(f"ERROR in POC A: {e}")
    raise
# --- End POC A Code ---


POC A: Loading Sentence Transformer: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
Using device: cpu
Embedding model loaded successfully.

Encoding 3 sample sentences...
Embeddings generated.
Shape: (3, 384)
Type: <class 'numpy.ndarray'>
First 5 dims of first embedding: [ 0.18921489  0.05337591 -0.06780824 -0.24332428  0.14563532]


In [13]:
!pip install faiss-cpu numpy -q
# faiss-gpu available but faiss-cpu simpler for POC

#Action - JIT Learning (AI Bot - Gemini):** "Basic `faiss-cpu` code for `IndexFlatL2`?" "How to `.add()` numpy vectors?" "How to `.search()`?" "Interpret FAISS search output `distances`, `indices`?"

In [14]:
# --- POC B Code (Colab) ---
# Make sure this cell runs AFTER the cell for POC A

import faiss
import numpy as np

# Requires 'embeddings_array' (NumPy) and 'embedding_model' from Colab POC A
if 'embeddings_array' not in locals() or not isinstance(embeddings_array, np.ndarray):
    print("ERROR: Run POC A first to generate 'embeddings_array'.")
    raise NameError("embeddings_array missing")
if 'embedding_model' not in locals():
    print("ERROR: Run POC A first to load 'embedding_model'.")
    raise NameError("embedding_model missing")

try:
    d = embeddings_array.shape[1] # Vector dimension
    print(f"POC B: Vector dimension (d): {d}")

    # 1. Create FAISS index
    index = faiss.IndexFlatL2(d)
    print(f"FAISS IndexFlatL2 (CPU) created.") # Assuming CPU install for now

    # 2. Add vectors (must be float32)
    index.add(embeddings_array.astype('float32'))
    print(f"Embeddings added. Index size: {index.ntotal}")

    # 3. Create Query Vector
    query_text = ["Bagaimana cara pulangkan barang Shopee?"]
    query_vector = embedding_model.encode(query_text).astype('float32')
    print(f"\nEncoded query: '{query_text[0]}'")

    # 4. Search
    k = 2 # Find top 2 neighbors
    print(f"Searching for top {k} neighbors...")
    distances, indices = index.search(query_vector, k)

    print("\nSearch Results:")
    print(f"Indices: {indices}")
    print(f"Distances: {distances}")

except Exception as e:
    print(f"ERROR during FAISS POC (Colab): {e}")
    raise
# --- End POC B Code ---

POC B: Vector dimension (d): 384
FAISS IndexFlatL2 (CPU) created.
Embeddings added. Index size: 3

Encoded query: 'Bagaimana cara pulangkan barang Shopee?'
Searching for top 2 neighbors...

Search Results:
Indices: [[2 0]]
Distances: [[19.052498 19.441292]]


In [16]:
!pip install -U langchain-community -q
!pip install transformers datasets torch sentencepiece pandas accelerate tqdm scikit-learn huggingface_hub ipywidgets joblib langchain sentence-transformers faiss-cpu langchain-huggingface -q
# Added langchain-huggingface
print("Dependencies installed/updated.")

Dependencies installed/updated.


In [17]:
#colab
import os
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import torch # Ensure torch is imported for device checks later
import pandas as pd
import numpy as np


print("\n--- Step 2.1: Load & Chunk Knowledge Base ---")
# Define path ON GOOGLE DRIVE relative to notebook location (or use full path)
kb_dir = '/content/drive/MyDrive/Colab_Notebooks/malay-qa-bot-rag/knowledge_base'
print(f"Loading documents from: {kb_dir}")


if not os.path.isdir(kb_dir):
     print(f"ERROR: Knowledge base directory '{kb_dir}' not found. Please create it and add text files.")
     raise FileNotFoundError(f"Directory not found: {kb_dir}")


try:
    # Load all .txt files from the directory, using UTF-8 encoding
    loader = DirectoryLoader(kb_dir, glob="**/*.txt", loader_cls=TextLoader, loader_kwargs={'encoding': 'utf-8'}, show_progress=True)
    documents = loader.load()


    if not documents:
        print("WARNING: No documents found in the directory.")
        docs = [] # Ensure 'docs' exists even if empty
    else:
        print(f"Loaded {len(documents)} document(s).")
        # print(f"Sample Doc 0 Content (first 300 chars):\n{documents[0].page_content[:300]}\n...")


        # Define the text splitter
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,  # Max characters per chunk (adjust as needed)
            chunk_overlap=150   # Overlap helps maintain context between chunks
        )
        print("\nSplitting documents into chunks...")
        # Split the loaded documents
        docs = text_splitter.split_documents(documents)
        print(f"Split into {len(docs)} chunks.")
        if docs:
             # print("\nSample Chunk 0 Content:")
             # print(docs[0].page_content)
             # print(f"\nSample Chunk 0 Metadata: {docs[0].metadata}")
             pass # Avoid printing too much
        else:
             print("No chunks created.")


except Exception as e:
    print(f"Error loading or splitting documents: {e}")
    raise


--- Step 2.1: Load & Chunk Knowledge Base ---
Loading documents from: /content/drive/MyDrive/Colab_Notebooks/malay-qa-bot-rag/knowledge_base
ERROR: Knowledge base directory '/content/drive/MyDrive/Colab_Notebooks/malay-qa-bot-rag/knowledge_base' not found. Please create it and add text files.


FileNotFoundError: Directory not found: /content/drive/MyDrive/Colab_Notebooks/malay-qa-bot-rag/knowledge_base

In [18]:
# step 2.2
from langchain_huggingface import HuggingFaceEmbeddings # Re-import for clarity if needed
import torch # Ensure torch is imported

print("\n--- Step 2.2: Load Embedding Model via LangChain ---")

embedding_model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
print(f"Loading embedding model: {embedding_model_name}")
try:
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    # LangChain wrapper for Sentence Transformers
    embeddings = HuggingFaceEmbeddings(
        model_name=embedding_model_name,
        model_kwargs={'device': device}, # Specify device
        encode_kwargs={'normalize_embeddings': False} # Keep default behaviour unless normalization needed
    )
    print(f"LangChain embedding model wrapper loaded onto device: {device}")
except Exception as e:
    print(f"ERROR loading embedding model via LangChain: {e}")
    raise


--- Step 2.2: Load Embedding Model via LangChain ---
Loading embedding model: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
LangChain embedding model wrapper loaded onto device: cpu


You successfully modified the import to from langchain_huggingface import HuggingFaceEmbeddings.
You re-ran the cell for Sub-Step 2.2.
The LangChainDeprecationWarning is now gone.
The HuggingFaceEmbeddings object (using the new import) loaded correctly and is utilizing the Colab GPU (cuda).


In [19]:
#step 2.3
from langchain.vectorstores import FAISS # Re-import for clarity
print("\n--- Step 2.3: Create & Save FAISS Vector Store ---")


# Ensure required variables exist
if not 'docs' in locals(): raise NameError("'docs' (chunked documents) not found.")
if not docs: print("WARNING: No document chunks found to index. FAISS store will be empty.")
if not 'embeddings' in locals(): raise NameError("'embeddings' model wrapper not found.")


# Define path on Google Drive to save the index
index_save_path = "/content/drive/MyDrive/Colab_Notebooks/malay-qa-bot-rag/faiss_malay_ecommerce_kb_index"
print(f"FAISS index save path: {index_save_path}")


try:
    if docs: # Only create if there are documents to index
        print(f"Creating FAISS index from {len(docs)} chunks (this may take time)...")
        # Creates embeddings and builds index in one step
        vectorstore = FAISS.from_documents(docs, embeddings)
        print("FAISS index created in memory.")


        # Save the index to Google Drive
        vectorstore.save_local(index_save_path)
        print(f"FAISS index saved locally to: {index_save_path}")


        # Optional: Verify saving by loading back
        print("\nVerifying saved index by reloading...")
        vectorstore_loaded = FAISS.load_local(
            index_save_path,
            embeddings,
            allow_dangerous_deserialization=True # Usually required when embeddings aren't standard TF/PyTorch
        )
        print(f"Successfully loaded index. Contains {vectorstore_loaded.index.ntotal} vectors.")
    else:
        print("Skipping FAISS index creation as no document chunks were found.")
        vectorstore = None # Ensure variable exists but is None


except Exception as e:
    print(f"ERROR during FAISS creation/saving/loading: {e}")
    raise


--- Step 2.3: Create & Save FAISS Vector Store ---


NameError: 'docs' (chunked documents) not found.

Analysis of Output:
--- Step 2.3: Create & Save FAISS Vector Store ---: Your print statement indicating the start of the step.
FAISS index save path: /content/drive/MyDrive/.../faiss_malay_ecommerce_kb_index: Shows the correct path on your Google Drive where the index will be saved.
Creating FAISS index from 25 chunks (this may take time)...: Indicates it started processing the docs list (which contained 25 chunks derived from your KB files). The FAISS.from_documents(docs, embeddings) function was called. This step involved:
Calling embeddings.embed_documents(list_of_chunk_texts) which ran the Sentence Transformer model (on GPU) to get 25 vectors.
Building the FAISS IndexFlatL2 structure in memory.
Adding the 25 vectors to the index.
FAISS index created in memory.: Confirms the in-memory index was built.
FAISS index saved locally to: /content/drive/MyDrive/...: Confirms the vectorstore.save_local(index_save_path) command completed, writing the index.faiss and index.pkl files to your Google Drive.
Verifying saved index by reloading...: Your optional verification step started.
Successfully loaded index. Contains 25 vectors.: Confirms that FAISS.load_local(...) worked correctly using the saved files and the embeddings object, and the reloaded index (vectorstore_loaded) contains the expected number of vectors (25).

In [14]:
#step 2.4 : implement rag chainfrom langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline
from langchain.vectorstores import FAISS # Re-import if needed
from langchain.chains import RetrievalQA

print("\n--- Step 2.4: Implement RAG Chain ---")


# --- 1. Load Saved FAISS Index ---
index_load_path = "/content/drive/MyDrive/Colab_Notebooks/malay-qa-bot-rag/faiss_malay_ecommerce_kb_index" # Path where index was saved
print(f"Loading saved FAISS index from: {index_load_path}")
try:
    if not 'embeddings' in locals(): raise NameError("'embeddings' not found.")
    # Load the vector store
    vectorstore = FAISS.load_local(index_load_path, embeddings, allow_dangerous_deserialization=True)
    print(f"FAISS index loaded. Contains {vectorstore.index.ntotal} vectors.")
except Exception as e:
    print(f"ERROR loading FAISS index: {e}. Ensure it was saved correctly in the previous step.")
    raise


# --- 2. Setup Retriever ---
# Convert the vector store into a retriever component
# search_kwargs controls how many documents (k) to fetch
retriever = vectorstore.as_retriever(search_kwargs={"k": 3}) # Get top 3 relevant chunks
print(f"\nRetriever created. Will retrieve k={retriever.search_kwargs.get('k', 'default')} documents.")


# --- 3. Setup LLM Pipeline (Using mt5-small for generative QA) ---
# You can experiment with other models later
llm_checkpoint = "google/mt5-small"
print(f"\nLoading LLM for generation: {llm_checkpoint}")
try:
    llm_tokenizer = AutoTokenizer.from_pretrained(llm_checkpoint)
    llm_model = AutoModelForSeq2SeqLM.from_pretrained(llm_checkpoint)
    device = 0 if torch.cuda.is_available() else -1
    # Create a text-generation pipeline from Transformers library
    pipe = pipeline(
        "text2text-generation", # Task for T5/BART models
        model=llm_model,
        tokenizer=llm_tokenizer,
        max_new_tokens=150, # Limit generated answer length
        device=device,
        # Add other pipeline args if needed, e.g., temperature for creativity
    )
    # Wrap the HF pipeline in LangChain's LLM abstraction
    llm = HuggingFacePipeline(pipeline=pipe)
    print(f"LLM pipeline created successfully on device {device}.")
except Exception as e:
    print(f"ERROR loading LLM or creating pipeline: {e}")
    raise


# --- 4. Create RetrievalQA Chain ---
# This chain combines the retriever and the LLM
chain_type = "stuff" # Simplest method: just "stuff" all retrieved docs into the prompt
print(f"\nCreating RetrievalQA chain with chain_type='{chain_type}'...")
try:
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type=chain_type,
        retriever=retriever,
        return_source_documents=True # Set to True to see which chunks were used
    )
    print("RetrievalQA chain created successfully.")
except Exception as e:
    print(f"ERROR creating RetrievalQA chain: {e}")
    raise


--- Step 2.4: Implement RAG Chain ---
Loading saved FAISS index from: /content/drive/MyDrive/Colab_Notebooks/malay-qa-bot-rag/faiss_malay_ecommerce_kb_index
FAISS index loaded. Contains 24 vectors.

Retriever created. Will retrieve k=3 documents.

Loading LLM for generation: google/mt5-small


/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:559: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Device set to use cuda:0


LLM pipeline created successfully on device 0.

Creating RetrievalQA chain with chain_type='stuff'...
RetrievalQA chain created successfully.


Analysis of Output:
--- Step 2.4: Implement RAG Chain ---: Your marker.
Loading saved FAISS index... FAISS index loaded. Contains 25 vectors.: Confirms the FAISS.load_local(...) function worked correctly, loading the index previously saved to your Drive.
Retriever created. Will retrieve k=3 documents.: Confirms vectorstore.as_retriever(search_kwargs={"k": 3}) executed successfully, creating the retriever component configured to fetch the top 3 most similar document chunks.
Loading LLM for generation: google/mt5-small: Indicates the start of loading the generative model.
/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:559: UserWarning: The sentencepiece tokenizer... uses the byte fallback option... not implemented in the fast tokenizers...:
What it is: This is another informational UserWarning from the transformers library. The original tokenizer for mt5-small (based on SentencePiece) has a feature to handle unknown characters by breaking them down into individual byte tokens. The "fast" version of the tokenizer (implemented in Rust for speed, which AutoTokenizer often loads by default) doesn't support this specific byte fallback feature.
Implication: In rare cases with unusual characters not in the tokenizer's vocabulary, the fast tokenizer might output an "unknown" token (<unk>) where the original SentencePiece tokenizer might have produced a sequence of byte tokens. For standard Malay text, this is unlikely to be a significant issue.
Action Needed: None. You can safely ignore this warning for this project.
Device set to use cuda:0: Confirms the check for GPU availability within the LLM loading code worked.
LLM pipeline created successfully on device 0.: Confirms that the transformers.pipeline("text2text-generation", ...) was created and the langchain.llms.HuggingFacePipeline wrapper (llm) was instantiated successfully, utilizing the GPU (device 0).
Creating RetrievalQA chain with chain_type='stuff'...: Indicates the start of the final chain creation.
RetrievalQA chain created successfully.: Success! Confirms RetrievalQA.from_chain_type(...) executed correctly, linking the loaded llm and retriever together into the qa_chain object.

In [15]:
#Step 2.5: Test RAG Pipeline
import time # Import time for timing
print("\n--- Step 2.5: Test RAG Pipeline ---")


# Ensure qa_chain exists
if not 'qa_chain' in locals(): raise NameError("'qa_chain' not defined.")


# --- Ask a question relevant to YOUR E-COMMERCE KB ---
test_question = "Berapa lama tempoh pemulangan untuk LazMall?" # Example, use your own!
print(f"\nTesting QA chain with question: '{test_question}'")


try:
    start_time = time.time()
    # Invoke the chain by passing a dictionary with the query
    result = qa_chain({"query": test_question})
    end_time = time.time()


    print("\n--- RAG Chain Result ---")
    print(f"Question: {result.get('query', 'N/A')}")
    # The 'result' key usually holds the final generated answer
    print(f"\nGenerated Answer: {result.get('result', 'ERROR: No answer generated.')}")
    print(f"\nTime taken: {end_time - start_time:.2f} seconds")


    # Display the source documents that the retriever found
    print("\nSource Documents Retrieved:")
    source_docs = result.get('source_documents', [])
    if source_docs:
         for i, doc in enumerate(source_docs):
             print(f"--- Source {i+1} (From: {doc.metadata.get('source', 'N/A')}) ---")
             # Display content snippet - adjust length as needed
             print(doc.page_content[:400] + "...")
             print("-------------------------")
    else:
         print("No source documents were returned by the retriever.")


except Exception as e:
    print(f"ERROR running QA chain: {e}")
    # Check for errors like context length exceeding model limit if using 'stuff'


--- Step 2.5: Test RAG Pipeline ---

Testing QA chain with question: 'Berapa lama tempoh pemulangan untuk LazMall?'


<ipython-input-15-e5c12054642c>:18: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": test_question})



--- RAG Chain Result ---
Question: Berapa lama tempoh pemulangan untuk LazMall?

Generated Answer: <extra_id_0> hanya boleh digunakan.

Time taken: 0.73 seconds

Source Documents Retrieved:
--- Source 1 (From: /content/drive/MyDrive/Colab_Notebooks/malay-qa-bot-rag/knowledge_base/lazada_content_malay .txt) ---
Ketahui cara membatalkan pesanan anda daripada akaun Lazada anda [disini].

2. **Apakah jenis Polisi Pemulangan yang tersedia di Lazada?**

Sila ambil perhatian bahawa bermula dari 31 Oktober 2024, tempoh pemulangan pelanggan akan dikemas kini dari 7 hari hingga 15 hari untuk Pasaran Lazada & Penjual LazGlobal.

Terdapat 3 jenis Polisi Pemulangan yang tersedia di Lazada, bergantung pada produk dan...
-------------------------
--- Source 2 (From: /content/drive/MyDrive/Colab_Notebooks/malay-qa-bot-rag/knowledge_base/shopee_content_malay -.txt) ---
⚠️ Nota:  

 Kadangkala, anda mungkin menghadapi masalah teknikal semasa mengemukakan permohonan Pulangan/Bayaran Balik, yang mungkin 

Analysis of Output:
--- Step 2.5: Test RAG Pipeline ---: Start marker.
Testing QA chain with question: 'Berapa lama tempoh pemulangan untuk LazMall?': Confirms the question being asked.
LangChainDeprecationWarning: The method \Chain.call` was deprecated... Use :meth:`~invoke` instead.`:
What: Another deprecation warning from LangChain. The way you called the chain (qa_chain({"query": ...})) uses the older __call__ method. The newer, preferred way is to use qa_chain.invoke({"query": ...}) or qa_chain.invoke(test_question) depending on the chain's expected input structure (often just the string query works for invoke).
Action Needed: None required for functionality now, but for future compatibility, update the call to use .invoke():
# OLD: result = qa_chain({"query": test_question})
# NEW:
result = qa_chain.invoke(test_question) # Often works directly with string
# OR more explicitly if needed:
# result = qa_chain.invoke({"query": test_question})
Use code with caution.
Python
You can make this change now or during Phase 2 refinement.
--- RAG Chain Result ---: Start of the results.
Question: Berapa lama tempoh pemulangan untuk LazMall?: Correctly repeats the input question.
Generated Answer: <extra_id_0> hanya boleh digunakan.:
Analysis: This answer is completely irrelevant and nonsensical. The <extra_id_0> is a special token often used by T5-based models during training (related to denoising objectives) that sometimes leaks into generation if the model is not perfectly fine-tuned or if the prompt/context confuses it. The rest of the phrase "hanya boleh digunakan" (only can be used) is also irrelevant.
Conclusion: The LLM (google/mt5-small) failed to synthesize a meaningful answer from the provided context for this specific question.
Time taken: 0.81 seconds: Shows the query processing time (mostly LLM inference on the GPU) was fast.
Source Documents Retrieved:: Shows the top 3 chunks retrieved by FAISS based on the question embedding.
Source 1 (lazada_content_malay .txt): Contains text about general return policy updates ("tempoh pemulangan pelanggan akan dikemas kini...") and the 3 types of policies. Somewhat relevant but doesn't directly state the LazMall duration.
Source 2 (shopee_content_malay -.txt): Contains notes about technical issues and return buttons for Shopee. Irrelevant.
Source 3 (lazada_content_malay .txt): Contains the table Tempoh Pemulangan Standard which explicitly shows "LazMall & Choice (selepas 1 Februari 2024) | 30 Hari". This is the highly relevant chunk containing the correct answer.
Retriever Performance: The retriever (FAISS + embedding model) successfully found the most relevant chunk containing the answer (Source 3). However, it also pulled in a somewhat relevant chunk (Source 1) and an irrelevant one (Source 2). This is typical - retrieval is often imperfect.
Generator (LLM) Performance: The LLM (mt5-small) failed to extract or synthesize the correct answer ("30 Hari") even though it was present in the retrieved context (Source 3). It produced unrelated output instead.
Overall Conclusion for Step 2.5:
The RAG pipeline executed successfully end-to-end without crashing.
The retriever component worked reasonably well, identifying the chunk containing the correct answer within its top 3 results.
The generator (LLM) component performed poorly for this query, failing to utilize the provided context effectively and generating nonsensical output.
Why the Poor Answer? (Possible Reasons):
Base LLM Capability: google/mt5-small is a relatively small model. While multilingual, its ability to perform precise QA synthesis based purely on context provided in a prompt might be limited without specific instruction fine-tuning for QA or summarization tasks.
Prompting (Implicit in RetrievalQA): The default prompt used by the stuff chain type might not be optimal for instructing mt5-small on how to extract the answer from the combined context chunks.
Context "Noise": Including less relevant chunks (Source 1, Source 2) alongside the correct one (Source 3) might have confused the small LLM.
What's Next (Despite Poor Answer Quality):
For the Level 2 MVP: The goal was a functional RAG pipeline. You achieved this – the pipeline runs, retrieves documents, and calls the LLM. The quality of the answer is an area for enhancement later.
Proceed to: Sub-Step 2.6: Commit Level 2 Work. Save the notebook and commit the code for the functional (though imperfect) RAG pipeline.
Address in Phase 2 / Level 4 Enhancements: Make notes to revisit LLM choice (try larger models, maybe the fine-tuned L1 model if adapted, or other HF models), prompt engineering, and potentially adding a re-ranker (Enhancement 4.1) to improve the quality of context fed to the LLM.


In [16]:
# In Colab code cell
!pip freeze > requirements.txt
# Manually edit .gitignore using Colab editor if needed:
# - Ensure faiss_malay_ecommerce_kb_index/ IS NOT ignored (we want to commit the small index)
# - Add knowledge_base/ if you DON'T want to commit the raw text files (optional)
print("requirements.txt updated. Check .gitignore for FAISS index.")

requirements.txt updated. Check .gitignore for FAISS index.


In [17]:
# !git config ... user info if needed
# Stage notebook, requirements, gitignore, AND the FAISS index folder
!git config --global user.email "amirulhazym@gmail.com"
!git config --global user.name "amirulhazym"
!git add 02-RAG-Pipeline.ipynb requirements.txt .gitignore faiss_malay_ecommerce_kb_index/
!git commit -m "Feat(P3L2): Implement RAG pipeline with LangChain and FAISS"
# !git push ... using PAT method

[main 4cbab6c] Feat(P3L2): Implement RAG pipeline with LangChain and FAISS
 4 files changed, 17 insertions(+), 143 deletions(-)
 rewrite 02-RAG-Pipeline.ipynb (97%)
 rewrite faiss_malay_ecommerce_kb_index/index.faiss (72%)


# **LEVEL 3**


In [ ]:
!pip freeze > requirements.txt